In [28]:
from pprint import pprint
import sys
import os

In [44]:
file="../CFG/graphite300K.corrected.cfg"
cfg_files = []

with open(file) as f:
    data = f.readlines()

for i,line in enumerate(data):
    # line = line.strip('\n')
    if "BEGIN_CFG" in line:
        cfg = [line]
        size = int(data[i+2].strip())
    elif "END_CFG" in line:
        cfg.append(line)
        cfg_files.append(cfg)
    else:
        try:
            cfg.append(line)
        except ValueError as e:
            print("You got a problem son")

In [45]:
len(cfg_files)

902

In [46]:
pprint(cfg_files[0])

['BEGIN_CFG\n',
 'Size\n',
 '108\n',
 'Supercell\n',
 '        7.392000    0.000000    0.000000\n',
 '       -3.696000    6.401660    0.000000\n',
 '        0.000000    0.000000   20.400000\n',
 'AtomData: id type cartes_x cartes_y cartes_z     fx     fy     fz\n',
 '           1    1   1.2280   0.7090   3.3480   0.1827   0.1054  -0.0150 \n',
 '           2    1   1.2280   0.7090  10.1480   0.1827   0.1054  -0.0150 \n',
 '           3    1   1.2280   0.7090  16.9480   0.1827   0.1054  -0.0150 \n',
 '           4    1  -0.0040   2.8429   3.3480   0.1827   0.1054  -0.0150 \n',
 '           5    1  -0.0040   2.8429  10.1480   0.1827   0.1054  -0.0150 \n',
 '           6    1  -0.0040   2.8429  16.9480   0.1827   0.1054  -0.0150 \n',
 '           7    1  -1.2360   4.9768   3.3480   0.1827   0.1054  -0.0150 \n',
 '           8    1  -1.2360   4.9768  10.1480   0.1827   0.1054  -0.0150 \n',
 '           9    1  -1.2360   4.9768  16.9480   0.1827   0.1054  -0.0150 \n',
 '          10    1   3

In [49]:
cfg_files[0][116]

'Energy\n'

In [33]:
size = int(cfg[2].strip())
element_type_dict = {0: "Li", 1: "C"}

if "AtomData" in cfg[7]:
    atoms = []
    for i in range(8, 8+size):
        atoms.append(cfg[i].strip().split())
    if len(atoms) != size:
        print("error, num atoms doesn't match size")
else:
    raise Exception("AtomData not found, check cfg format")
    
atom_data = {
    "size": len(atoms),
    "index": [],
    "element": [],
    "xyz": [],
    "forces": []
}

for at in atoms:
    atom_data["index"].append(int(at[0]))
    atom_data["element"].append(element_type_dict[int(at[1])])
    atom_data["xyz"].append([,float(at[3]),float(at[4])])
    atom_data["forces"].append([float(at[5]),float(at[6]),float(at[7])])

In [3]:
def read_cfgs(cfg_file):
    cfg_trj = []
    with open(cfg_file) as f:
        data = f.readlines()

    for i,line in enumerate(data):
        # line = line.strip('\n')
        if "BEGIN_CFG" in line:
            cfg = [line]
            size = int(data[i+2].strip())
        elif "END_CFG" in line:
            cfg.append(line)
            cfg_trj.append(cfg)
        else:
            try:
                cfg.append(line)
            except ValueError as e:
                print("You got a problem son")
    return cfg_trj

In [50]:
def cfg2data(cfg):
    size = int(cfg[2].strip())
    element_type_dict = {0: "Li", 1: "C"}

    if "AtomData" in cfg[7]:
        atoms = []
        for i in range(8, 8+size):
            atoms.append(cfg[i].strip().split())
        if len(atoms) != size:
            print("error, num atoms doesn't match size")
    else:
        raise Exception("AtomData not found, check cfg format")
        
    if "Energy" in cfg[8 + size]:
        energy = cfg[9 + size]

    atom_data = {
        "size": len(atoms),
        "index": [],
        "element": [],
        "xyz": [],
        "forces": [],
        "energy": energy
    }

    for at in atoms:
        atom_data["index"].append(int(at[0]))
        atom_data["element"].append(element_type_dict[int(at[1])])
        atom_data["xyz"].append([float(at[2]),float(at[3]),float(at[4])])
        atom_data["forces"].append([float(at[5]),float(at[6]),float(at[7])])
        
    return atom_data

In [5]:
graphite_LiC6 = ['../CFG/graphite300K.corrected.cfg',
                 '../CFG/graphite600K.corrected.cfg',
                 '../CFG/graphite900K.corrected.cfg',
                 '../CFG/LiC6.300K.cfg',
                 '../CFG/LiC6.600K.cfg',
                 '../CFG/LiC6.900K.cfg']

In [51]:
graphite300K_trj = read_cfgs('../CFG/graphite300K.corrected.cfg')
graphite300K_xyz = [cfg2data(cfg) for cfg in graphite300K_trj]

In [9]:
# len(graphite300K_trj)
len(graphite300K_xyz)

902

In [21]:
with open("graphite300K.xyz", "w") as f:
    for frame in graphite300K_xyz:
        f.write(f"{frame['size']}\n")
        f.write(f"graphite\n")
        # f.write(f"")
        # original_stdout = sys.stdout
        # sys.stdout = f
        for i in range(frame['size']):
            x = round(frame['xyz'][i][0],4)
            y = round(frame['xyz'][i][1],4)
            z = round(frame['xyz'][i][2],4)
            f.write(f"{frame['element'][i]}\t{x:.4f}\t{y:.4f}\t{z:.4f}\n")
        #     print(*atom_data['xyz'][i], sep="\t")
        # sys.stdout = original_stdout

In [22]:
def write_xyz_trajectory(atom_data_dict,name):
    with open(f"{name}.xyz", "w") as f:
        for frame in atom_data_dict:
            f.write(f"{frame['size']}\n")
            f.write(f"graphite\n")

            for i in range(frame['size']):
                x = round(frame['xyz'][i][0],4)
                y = round(frame['xyz'][i][1],4)
                z = round(frame['xyz'][i][2],4)
                f.write(f"{frame['element'][i]}\t{x:.4f}\t{y:.4f}\t{z:.4f}\n")

In [26]:
def write_xyz_from_cfg(cfg_file, cfg_name):
    cfgs = read_cfgs(cfg_file)
    xyz = [cfg2data(cfg) for cfg in cfgs]
    write_xyz_trajectory(xyz, cfg_name)

In [23]:
graphite600K = read_cfgs('../CFG/graphite600K.corrected.cfg')
gr600Kxyz = [cfg2data(cfg) for cfg in graphite600K]
write_xyz_trajectory(gr600Kxyz, "graphite600K")

In [24]:
graphite900K = read_cfgs('../CFG/graphite900K.corrected.cfg')
gr900Kxyz = [cfg2data(cfg) for cfg in graphite900K]
write_xyz_trajectory(gr900Kxyz, "graphite900K")

In [25]:
lic6300K = read_cfgs('../CFG/LiC6.300K.cfg')
lic6600K = read_cfgs('../CFG/LiC6.600K.cfg')
lic6900K = read_cfgs('../CFG/LiC6.900K.cfg')

lic6_300K_xyz = [cfg2data(cfg) for cfg in lic6300K]
lic6_600K_xyz = [cfg2data(cfg) for cfg in lic6600K]
lic6_900K_xyz =  [cfg2data(cfg) for cfg in lic6900K]

write_xyz_trajectory(lic6_300K_xyz, "lic6_300K")
write_xyz_trajectory(lic6_600K_xyz, "lic6_600K")
write_xyz_trajectory(lic6_900K_xyz, "lic6_900K")

In [30]:
os.listdir('../CFG')

['.ipynb_checkpoints',
 'LiC12.300K.cfg',
 'LiC12.600K.cfg',
 'LiC12.900K.cfg',
 'LiC18.300K.cfg',
 'LiC6.300K.cfg',
 'LiC6.600K.cfg',
 'LiC6.900K.cfg',
 'graphite300K.corrected.cfg',
 'graphite600K.corrected.cfg',
 'graphite900K.corrected.cfg',
 'test_structures',
 'wrong_tag_graphite']

In [38]:
test = my_list[0].strip(".cfg")
test.strip("/CFG/")

'LiC12.300K'

In [40]:
my_list  =  ['../CFG/LiC12.300K.cfg',
             '../CFG/LiC12.600K.cfg',
             '../CFG/LiC12.900K.cfg',
             '../CFG/LiC18.300K.cfg']
for i in my_list:
    name = i.strip(".cfg")
    name = name.strip("/CFG/")
    write_xyz_from_cfg(i,name)

In [47]:
training_set = []
for file in graphite_LiC6:
    cfgs = read_cfgs(file)
    training_set += cfgs

In [48]:
len(training_set)

7038

In [18]:
atom_data = graphite300K_xyz[0]

In [20]:
with open("test.xyz", "w") as f:
    f.write(f"{atom_data['size']}\n\n")
    # f.write(f"")
    # original_stdout = sys.stdout
    # sys.stdout = f
    for i in range(atom_data['size']):
        x = round(atom_data['xyz'][i][0],4)
        y = round(atom_data['xyz'][i][1],4)
        z = round(atom_data['xyz'][i][2],4)
        f.write(f"{atom_data['element'][i]}\t{x:.4f}\t{y:.4f}\t{z:.4f}\n")
    #     print(*atom_data['xyz'][i], sep="\t")
    # sys.stdout = original_stdout

In [28]:
atom_data['xyz'][0]

[1.228, 0.709, 3.348]

In [31]:
print(*atom_data['xyz'][0], sep="\t")

1.228	0.709	3.348
